# Notebook #1 (traceroute probes result)

## Get result from Atlas nodes

In [1]:
import pandas as pd
import numpy as np
import re
import IP2Location
import pytz
import time

from datetime import datetime
from ripe.atlas.cousteau import AtlasResultsRequest
from ripe.atlas.cousteau import ProbeRequest
import ipaddress

# Functions

In [2]:
# return a dic with all the probe infos
def get_probes_info(probes):

    result = []

    for probe in probes:

        if (probe["address_v4"] and probe["address_v6"]):
            record = {
                "address_v4": probe["address_v4"],
                "address_v6": probe["address_v6"],
                "probe_id"   : probe["id"],
                'cc'        : probe["country_code"],
                #'geometry'  : probe['geometry']['coordinates'],
                'asn_v4'    : probe['asn_v4'],
                'asn_v6'    : probe['asn_v6']

            }
            result.append(record)
    return result

In [3]:
# split an list in smaller lists
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [4]:
## Get the results from atlas
### probes = list of atlas probes ID
### start = timestamp
### stop timestamp
### measurement_id = ID of the task
def get_atlas_results(probes_id_list, start, stop, measurement_id):

    measurement_results = []

    #print ("star: {} stop: {} measurement_id {}".format(start, stop, measurement_id))
    # split the probe_id
    chuncked_list = list(chunks(probes_id_list, 500))

    # for each chunk of probes_id do
    for small_list in (chuncked_list):

        kwargs = {"msm_id"   : measurement_id,
                  "start"    : datetime.utcfromtimestamp(start),
                  "stop"     : datetime.utcfromtimestamp(stop),
                  "probe_ids": small_list}

        is_success, results = AtlasResultsRequest(**kwargs).create()

        if is_success:
             measurement_results = results + measurement_results

    return (measurement_results)


In [5]:
## Parse the results from Atlas probes (traceroute)
def parse_traceroute_results(all_results):

    # initial values
    prob_id = ""
    measurement_id  = ""
    origin = ""
    af = ""
    ip_path = []

    traceroute_records = []
    # evaluate all the lines from the list of results.
    # each line represent an traceroute output performed by the probe
    for result in (all_results):


        prob_id = result['prb_id']
        measurement_id = result['msm_id']

        # if "NEWLINE" is present, means that it's not a json (legacy format)
        # we have to parse this without use the Sagan lib
        str_result = str(result)
        if (str_result.find("NEWLINE") != -1):
            path = []
            try:
                data = result['src_addr'].split("NEWLINE")
            except:
                data = result['result'].split("NEWLINE")

            # remove headers at the first line
            data.pop(0)

            # iterate for every line to get the hops
            for line  in (data):

                # line not empty
                line = line.rstrip()
                if not len(line.strip()) == 0:

                    # second element in the line
                    try:
                        ip = line.split()[1]
                    except IndexError:
                        ip = "None"

                    # is this second element an IP?
                    if (ip):
                        try:
                            ip_addr = ipaddress.ip_address(ip)
                        except ValueError:
                            # ip not valid
                            ip="None"

                    # append value
                    path.append(ip)

            # proto identify (SHAME ON ME) FIX
            if any(":" in s for s in path):
              af = 6
            else:
              af = 4

            if (len(path)>1):
                origin = path[0] or None
                ip_path = path

        # parse JSON format
        else:
            
            ip_path = []
            if not (result.get('result')):
                continue

            if (result.get('from')):
                origin = result['from']

            hops = result['result']

            if not isinstance(hops, list):
                continue

            for hop in (hops):

                if not (hop.get('result')):
                   ip_path.append("None")
                   next
                # first line result from traceroute
                else :
                    ip = hop['result'][0].get('from')
                    if (ip):
                        ip_path.append(ip)
                    else:
                       ip_path.append("None")

            # proto identify (SHAME ON ME) FIX
            if any(":" in s for s in ip_path):
              af = 6
            else:
              af = 4

        # sumarize results
        record = {
            "probe_id" : prob_id,
            "hops"     : str(ip_path),
            "origin"   : origin,
            "measurement_id": measurement_id,
            "proto"     : af,
        }
        if (len(ip_path)>2):
            traceroute_records.append(record)

    # remove duplicates in the dic
    unique_elems = [dict(y) for y in set(tuple(x.items()) for x in traceroute_records)]

    return (unique_elems)

# Main

## Determine witch IDs are associated with the root-servers (probe_id)




### Root-Servers IDs on Atlas

In [6]:
# https://atlas.ripe.net/docs/built-in/
# IDs used by Atlas to identify the root-servers
root_map = {
    'a': '09',
    'b': '10',
    'c': '11',
    'd': '12',
    'e': '13',
    'f': '04',
    'g': '14',
    'h': '15',
    'i': '05',
    'j': '16',
    'k': '01',
    'l': '08',
    'm': '06',
}

## Dataframe with all measurement IDs

In [8]:
# create a dataframe with all the mesuareament IDs for each root-server letter
root_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']
#root_list = ['k', 'l', 'm']
result = []

for root in root_list:
    for proto in ["ipv4","ipv6"]:
        record = {
            "root_id": root,
            "proto": proto,
            "root" : root,
        }
        result.append(record)

df = pd.DataFrame.from_dict(result)
df = df.replace({"root_id": root_map})

# convert to respective identication code on RIPE ATLAS database
# https://atlas.ripe.net/docs/built-in/
# 50xx - IPv4 traceroute
# 60xx - IPv6 traceroute
df['mesuareament_id'] = df.apply(lambda row: 50 if (row['proto']) == ("ipv4") else 60, axis=1)
df["mesuareament_id"] = df["mesuareament_id"].astype(str).add(df["root_id"])
df

,root_id,proto,root,mesuareament_id
0,09,ipv4,a,5009
1,09,ipv6,a,6009
2,10,ipv4,b,5010
3,10,ipv6,b,6010
4,11,ipv4,c,5011
5,11,ipv6,c,6011
6,12,ipv4,d,5012
7,12,ipv6,d,6012
8,13,ipv4,e,5013
9,13,ipv6,e,6013


## RIPE ATLAS

## Find all the dual stack (IPv4/IPv6) probes for a list of countries

In [9]:
# set a list of countries 
# we want to compare the BGP info with traceroute CC
# then we only select nodes where we have a RIPE RIS collector (IXP)
filters = {"country_code__in":"AU,BR,CH,DE,ES,FR,IT,JP,NL,RU,SE,GB,US,ZA"}
# filters = {"country_code__in":"BR"}

# request the probe info
probes = ProbeRequest(**filters)
result = get_probes_info(probes)

In [10]:
# convert the result to a dataframe
df_probes = pd.DataFrame.from_dict(result)
df_probes.set_index("probe_id",inplace=True)

# unknown ASNs are set to "0"
df_probes['asn_v6'] = df_probes['asn_v6'].fillna(0).astype(int)
df_probes['asn_v4'] = df_probes['asn_v4'].fillna(0).astype(int)
df_probes.to_csv("probes.txt", sep=";")


## Probe_ID
### Dataframe with all the probes info from the filtered countries


In [15]:
df_probes.head()

,address_v4,address_v6,cc,asn_v4,asn_v6
probe_id,,,,,
3,62.251.17.184,2001:985:71fc:fd:220:4aff:fec8:2532,NL,3265,3265
4,83.163.50.165,2001:980:57a4:1:220:4aff:fec8:244a,NL,3265,3265
5,83.163.239.181,2001:981:602b:1:220:4aff:fec8:2355,NL,3265,3265
8,83.68.21.139,2001:984:aee9:6:220:4aff:fec8:2464,NL,3265,3265
9,193.0.0.78,2001:67c:2e8:ffe1:220:4aff:fec6:cc9d,NL,3333,3333


In [16]:
df_probes.count()

address_v4    5319
address_v6    5319
cc            5319
asn_v4        5319
asn_v6        5319
dtype: int64

In [18]:
# reload dataframe
df_probes = pd.read_csv("probes.txt", sep=";")
df_probes.set_index("probe_id",inplace=True)
df_probes.head()

,address_v4,address_v6,cc,asn_v4,asn_v6
probe_id,,,,,
3,62.251.17.184,2001:985:71fc:fd:220:4aff:fec8:2532,NL,3265,3265
4,83.163.50.165,2001:980:57a4:1:220:4aff:fec8:244a,NL,3265,3265
5,83.163.239.181,2001:981:602b:1:220:4aff:fec8:2355,NL,3265,3265
8,83.68.21.139,2001:984:aee9:6:220:4aff:fec8:2464,NL,3265,3265
9,193.0.0.78,2001:67c:2e8:ffe1:220:4aff:fec6:cc9d,NL,3333,3333


## number of probes dual-stack by country

In [22]:
df_probes['probe_id'] = df_probes.index
probes_id_list = df_probes['probe_id'].tolist()
print ("number of probes: {}".format(len(probes_id_list)))
df_probes['cc'].value_counts()

number of probes: 5319


DE    1418
US     944
FR     801
GB     497
NL     453
CH     266
RU     223
JP     167
AU     132
IT     123
SE     111
BR      80
ZA      52
ES      52
Name: cc, dtype: int64

## RIPE ATLAS 
###  Get results from atlas probes

In [28]:
importlib.reload(SAND)
utc = pytz.utc
start = datetime(2008, 3, 1, 1, 0, 0) # 2008-03-01
stop = datetime(datetime.today().year, datetime.today().month, 1, 1, 0, 0)

all_results  =  [] 
for  mesuareament_id in df['mesuareament_id'].unique().tolist():
    
    root = df[df["mesuareament_id"]==mesuareament_id]['root'].to_string(index=False)
#     print('Working on {}-Root'.format(root.upper()))
    
    cur_date = start
    
    appended_data = pd.DataFrame()
    
    while cur_date <= stop:
#         print("\t {} ".format(cur_date))
       
        date = "{}".format(cur_date).split(" ")[0]
        utc_dt = utc.localize(cur_date)
        timestamp = int(time.mktime(utc_dt.timetuple()))
        
        print ("{}, measurement_id: {}, timestamp: {}, root: {}".format(cur_date,mesuareament_id,timestamp,root)) 
            
        # time window to collect the results
        n_hours = 12
        start_probe_time = timestamp
        stop_probe_time = start_probe_time + (3600 * n_hours)
            
        results_from_probes = get_atlas_results(probes_id_list, start_probe_time, stop_probe_time, mesuareament_id)
        traceroute_results = parse_traceroute_results(results_from_probes)
        
        # results found
        if (len(traceroute_results)>0):
 
            df_traceroute_results = pd.DataFrame(traceroute_results)
            df_traceroute_results.set_index("probe_id",inplace=True)
            df_result = pd.concat([df_traceroute_results,df_probes], axis=1, join_axes=[df_traceroute_results.index])
            df_result['timestamp'] = start_probe_time
            df_result['root'] = root
            print ("number of results from probes: {}".format(len(df_result)))
            appended_data = pd.concat([appended_data, df_result])
            #print (len(appended_data))
            
        year = cur_date.year + 1 if cur_date.month == 12 else cur_date.year
        month = 1 if cur_date.month == 12 else cur_date.month + 1
        cur_date = datetime(year, month, 1, 1, 0, 0)    
        
    
    filename = "traceroute-results-root-{}-measurement_id-{}.txt".format(root,measurement_id)

    appended_data.to_csv(filename, sep=";")
        
        


NameError: name 'importlib' is not defined

In [21]:
# utc = pytz.utc
# start = datetime(2008, 3, 1, 1, 0, 0) # 2008-03-01
# stop = datetime(datetime.today().year, datetime.today().month, 1, 1, 0, 0)
import pprint
start_probe_time = 1320105600
start_probe_time = 1328054400

start_probe_time = 1510099200

stop_probe_time  = start_probe_time + (86400 * 2)


# results_from_probes = get_atlas_results(probes_id_list, start_probe_time, stop_probe_time, 6009)
results_from_probes = get_atlas_results(["1","2"], 1538352000, 1538438399, 5009)
pprint.pprint ((results_from_probes))


[{'af': 4,
  'dst_addr': '198.41.0.4',
  'dst_name': '198.41.0.4',
  'endtime': 1538353409,
  'from': '86.91.158.30',
  'fw': 4790,
  'lts': 46,
  'msm_id': 5009,
  'msm_name': 'Traceroute',
  'paris_id': 13,
  'prb_id': 2,
  'proto': 'UDP',
  'result': [{'hop': 1,
              'result': [{'from': '192.168.0.1',
                          'rtt': 1.898,
                          'size': 68,
                          'ttl': 64},
                         {'from': '192.168.0.1',
                          'rtt': 1.898,
                          'size': 68,
                          'ttl': 64},
                         {'from': '192.168.0.1',
                          'rtt': 1.847,
                          'size': 68,
                          'ttl': 64}]},
             {'hop': 2,
              'result': [{'from': '192.168.254.254',
                          'rtt': 3.347,
                          'size': 68,
                          'ttl': 63},
                         {'from': '192.168.2

                                                         'label': 716794,
                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 12.164,
                          'size': 140,
                          'ttl': 248}]},
             {'hop': 9,
              'result': [{'from': '64.125.21.22',
                          'rtt': 12.595,
                          'size': 28,
                          'ttl': 247},
                         {'from': '64.125.21.22',
                          'rtt': 12.071,
                          'size': 28,
                          'ttl': 247},
                         {'from': '64.125.21.22',
                          'rtt': 12.713,
                          'size': 28,
                 

                          'rtt': 1.656,
                          'size': 68,
                          'ttl': 64},
                         {'from': '192.168.199.1',
                          'rtt': 1.838,
                          'size': 68,
                          'ttl': 64}]},
             {'hop': 2,
              'result': [{'from': '194.109.5.177',
                          'rtt': 6.942,
                          'size': 28,
                          'ttl': 254},
                         {'from': '194.109.5.177',
                          'rtt': 6.235,
                          'size': 28,
                          'ttl': 254},
                         {'from': '194.109.5.177',
                          'rtt': 6.249,
                          'size': 28,
                          'ttl': 254}]},
             {'hop': 3,
              'result': [{'from': '194.109.7.69',
                          'rtt': 6.243,
                          'size': 28,
                          'ttl': 

                          'rtt': 12.333,
                          'size': 68,
                          'ttl': 55}]},
             {'hop': 11, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]},
             {'hop': 12, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]},
             {'hop': 13, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]},
             {'hop': 14, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]},
             {'hop': 15, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]},
             {'hop': 255, 'result': [{'x': '*'}, {'x': '*'}, {'x': '*'}]}],
  'size': 40,
  'src_addr': '192.168.199.102',
  'stored_timestamp': 1538361105,
  'timestamp': 1538360998,
  'type': 'traceroute'},
 {'af': 4,
  'dst_addr': '198.41.0.4',
  'dst_name': '198.41.0.4',
  'endtime': 1538362408,
  'from': '86.91.158.30',
  'fw': 4790,
  'lts': 15,
  'msm_id': 5009,
  'msm_name': 'Traceroute',
  'paris_id': 2,
  'prb_id': 2,
  'proto': 'UDP',
  'result': [{'hop': 1,
              'result': [{'from': '19

                          'ttl': 253},
                         {'from': '194.109.7.73',
                          'rtt': 6.218,
                          'size': 28,
                          'ttl': 253}]},
             {'hop': 4,
              'result': [{'from': '194.109.5.3',
                          'rtt': 6.79,
                          'size': 28,
                          'ttl': 252},
                         {'from': '194.109.5.3',
                          'rtt': 6.338,
                          'size': 28,
                          'ttl': 252},
                         {'from': '194.109.5.3',
                          'rtt': 6.523,
                          'size': 28,
                          'ttl': 252}]},
             {'hop': 5,
              'result': [{'from': '80.249.208.122',
                          'rtt': 6.844,
                          'size': 28,
                          'ttl': 251},
                         {'from': '80.249.208.122',
                        

...

 {'af': 4,
  'dst_addr': '198.41.0.4',
  'dst_name': '198.41.0.4',
  'endtime': 1538425410,
  'from': '86.91.158.30',
  'fw': 4790,
  'lts': 55,
  'msm_id': 5009,
  'msm_name': 'Traceroute',
  'paris_id': 5,
  'prb_id': 2,
  'proto': 'UDP',
  'result': [{'hop': 1,
              'result': [{'from': '192.168.0.1',
                          'rtt': 2.052,
                          'size': 68,
                          'ttl': 64},
                         {'from': '192.168.0.1',
                          'rtt': 1.741,
                          'size': 68,
                          'ttl': 64},
                         {'from': '192.168.0.1',
                          'rtt': 1.852,
                          'size': 68,
                          'ttl': 64}]},
             {'hop': 2,
              'result': [{'from': '192.168.254.254',
                          'rtt': 2.138,
                          'size': 68,
                          'ttl': 63},
                         {'from': '192.168.25

                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 12.323,
                          'size': 140,
                          'ttl': 244},
                         {'from': '64.125.29.17',
                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 701946,
                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 

                          'ttl': 251}]},
             {'hop': 6,
              'result': [{'from': '64.125.31.104',
                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 594280,
                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 12.335,
                          'size': 140,
                          'ttl': 243},
                         {'from': '64.125.31.104',
                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 594280,
               

                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 301216,
                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 24.254,
                          'size': 140,
                          'ttl': 251},
                         {'from': '134.222.48.205',
                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 301216,
                                                         's': 1,
                                                         'ttl': 1

                          'rtt': 12.525,
                          'size': 140,
                          'ttl': 248},
                         {'from': '64.125.30.235',
                          'icmpext': {'obj': [{'class': 1,
                                               'mpls': [{'exp': 0,
                                                         'label': 746496,
                                                         's': 1,
                                                         'ttl': 1}],
                                               'type': 1}],
                                      'rfc4884': 0,
                                      'version': 2},
                          'rtt': 12.585,
                          'size': 140,
                          'ttl': 248}]},
             {'hop': 9,
              'result': [{'from': '64.125.21.22',
                          'rtt': 12.451,
                          'size': 28,
                          'ttl': 247},
                  